In [1]:
# import 
import sys; 
if '../../modules/' not in sys.path: sys.path.append('../../modules/')
import myutils
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


%matplotlib inline

# fast.ai import
from fastai.imports import *
from fastai.structured import *

from sklearn.ensemble import RandomForestRegressor

In [2]:
path = '../../../datasets/grocery/'

In [3]:
!head -3 {path}train.csv

id,date,store_nbr,item_nbr,unit_sales,onpromotion
0,2013-01-01,25,103665,7.0,
1,2013-01-01,25,105574,1.0,


In [4]:
!ls {path}

holidays_events.csv  sample_submission.csv.7z  train.csv      transactions.csv
items.csv	     stores.csv		       Train.feather
oil.csv		     test.csv		       Train.pickle


In [5]:
!head -6 {path}items.csv {path}stores.csv {path}transactions.csv {path}oil.csv {path}holidays_events.csv

==> ../../../datasets/grocery/items.csv <==
item_nbr,family,class,perishable
96995,GROCERY I,1093,0
99197,GROCERY I,1067,0
103501,CLEANING,3008,0
103520,GROCERY I,1028,0
103665,BREAD/BAKERY,2712,1

==> ../../../datasets/grocery/stores.csv <==
store_nbr,city,state,type,cluster
1,Quito,Pichincha,D,13
2,Quito,Pichincha,D,13
3,Quito,Pichincha,D,8
4,Quito,Pichincha,D,9
5,Santo Domingo,Santo Domingo de los Tsachilas,D,4

==> ../../../datasets/grocery/transactions.csv <==
date,store_nbr,transactions
2013-01-01,25,770
2013-01-02,1,2111
2013-01-02,2,2358
2013-01-02,3,3487
2013-01-02,4,1922

==> ../../../datasets/grocery/oil.csv <==
date,dcoilwtico
2013-01-01,
2013-01-02,93.14
2013-01-03,92.97
2013-01-04,93.12
2013-01-07,93.2

==> ../../../datasets/grocery/holidays_events.csv <==
date,type,locale,locale_name,description,transferred
2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2012-04-12,Holiday,Local,Cuenca,Funda

In [6]:
#%%time
#df_all = pd.read_csv(f'{path}train.csv', date_parser=['date']) 

    <string>:2: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.

    CPU times: user 56.1 s, sys: 7.8 s, total: 1min 3s
    Wall time: 7min 15s

**To make this fast, better if can define the data type of columns at first hand, so that python have to doo less work**

In [7]:
types= {'id':'int64','store_nbr':'int8','item_nbr':'int32','unit_sales':'float32','onpromotion':'object'}

In [11]:
# %%time
# df_all = pd.read_csv(f'{path}train.csv', date_parser=['date'], dtype=types, low_memory=False, infer_datetime_format=True) 

CPU times: user 1min 55s, sys: 1min 32s, total: 3min 28s
Wall time: 34min 52s


In [20]:
# %%time
# df_train = pd.read_csv(f'{path}train.csv', parse_dates=['date'], dtype=types, low_memory=False, infer_datetime_format=True) 

CPU times: user 2min 53s, sys: 1min 40s, total: 4min 33s
Wall time: 1h 33min 46s


In [17]:
# pd.to_pickle(df_all, f'{path}Train.pickle')  #took less space

In [18]:
# df_all.to_feather(f'{path}Train.feather')  # not better compared with pickle

In [9]:
df_all = pd.read_pickle(f'{path}Train.pickle')